## JAXA AMSR2 DATA RETRIEVAL RECIPE

This script is used to download (37 and 89 GHz) passive microwave data from the GCOM-W1 JAXA satellite. It is based on an earlier script for retrieving AMSR-E AQUA/TERRA data from the NSIDC server.

In [1]:
%reload_ext autoreload
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.io.img_tiles import OSM
from cartopy.io.img_tiles import GoogleTiles as moa

import glob

import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr
import pyproj
import osr
import datetime
import subprocess
import shutil
import numpy as np
from ipywidgets import interact

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
os.path.abspath('/')

'C:\\'

In [ ]:
#https://gportal.jaxa.jp/download/standard/GCOM-W/GCOM-W.AMSR2/L1B/2/2018/08/GW1AM2_201808271242_064A_L1SGBTBR_2220220.h5
url_base = 'https://gportal.jaxa.jp/download/standard/GCOM-W/GCOM-W.AMSR2/'
url_folder = '{:s}/{:s}/'
url_template = url_base + url_folder + 'L3.TB{:s}GHz_{:s}/{:s}/{:s}/08/GW1AM2_{:s}_01D_EQMD_L3SGT{:s}{:s}{:s}2220220.h5'
download_template = 'wget --http-user={:s} --http-password={:s} --load-cookies mycookies.txt --save-cookies mycookies.txt --keep-session-cookies --no-check-certificate --auth-no-challenge -r --reject "index.html*" -np -e robots=off {:s}'
out_path = os.path.abspath('netcdf') #Can I call this when the file extention is .h5?
if not(os.path.isdir(out_path)):
    # prepare the output path
    os.makedirs(out_path)
    
freq = '36' # 36 GhZ following Brakenridge 2007 (WRR M/C ratio paper)
res = '3.125'  # can also be 12.5 or 25, although this depends on the chosen frequency as well
# freq = '36' # 36 GhZ following Brakenridge 2007 (WRR M/C ratio paper)
HV = 'V' # Horizontal or Vertical polarisation
AD = 'D' # 